<a href="https://colab.research.google.com/github/NinhDT22022522/NLP-ASM2/blob/main/vector_space_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.2 MB/s eta 0:00:00


In [25]:
from pyvi.ViTokenizer import tokenize
import re, os, string
import pandas as pd

def clean_text(text):
    text = re.sub('<.*?>', '', text).strip()
    text = re.sub('(\s)+', r'\1', text)
    return text

def normalize_text(text):
    listpunctuation = string.punctuation.replace('_', '')
    for i in listpunctuation:
        text = text.replace(i, ' ')
    return text.lower()

# List stopwords
filename = '/content/drive/MyDrive/Colab_Notebooks/NLP/Week2/stopwords.csv'
data = pd.read_csv(filename, sep="\t", encoding='utf-8')
list_stopwords = data['stopwords']

def remove_stopword(text):
    pre_text = []
    words = text.split()
    for word in words:
        if word not in list_stopwords:
            pre_text.append(word)
    text2 = ' '.join(pre_text)

    return text2

def sentence_segment(text):
    sents = re.split("([.?!])?[\n]+|[.?!] ", text)
    return sents

def word_segment(sent):
    sent = tokenize(sent)
    return sent

path_to_corpus = '/content/drive/MyDrive/Colab_Notebooks/NLP/Week2/AA/wiki_00'

f_w = open('/content/drive/MyDrive/Colab_Notebooks/NLP/Week2/datatrain.txt', 'w', encoding='utf-8')

with open(path_to_corpus, 'r', encoding='utf-8') as f_r:
    contents = f_r.read().strip().split('</doc>')
    for content in contents:
        if len(content) < 5:
            continue
        content = clean_text(content)
        sents = sentence_segment(content)
        for sent in sents:
            if sent:
                sent = word_segment(sent)
                sent = remove_stopword(normalize_text(sent))
                if len(sent.split()) > 1:
                    f_w.write(sent + '\n')

f_w.close()


In [36]:
from gensim.models import Word2Vec
import os

# Đường dẫn tới thư mục lưu trữ mô hình
model_dir = '/content/drive/MyDrive/Colab_Notebooks/NLP/Week2/model'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Đường dẫn tới dữ liệu huấn luyện
path_data = '/content/drive/MyDrive/Colab_Notebooks/NLP/Week2/datatrain.txt'

def read_data(path):
    train_data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            train_data.append(line.split())
    return train_data

if __name__ == '__main__':
    # Đọc dữ liệu
    train_data = read_data(path_data)

    # Huấn luyện mô hình Word2Vec
    model_w2v = Word2Vec(sentences=train_data, vector_size=150, window=10, min_count=2, workers=4, sg=0)

    # Lưu mô hình Word2Vec đầy đủ
    model_file = os.path.join(model_dir, "word2vec.model")
    try:
        model_w2v.save(model_file)
        print(f"Model saved successfully to {model_file}")
    except Exception as e:
        print(f"Error saving model: {e}")


Model saved successfully to /content/drive/MyDrive/Colab_Notebooks/NLP/Week2/model/word2vec.model


In [32]:
from gensim.models import FastText
import os

# Đường dẫn tới thư mục lưu trữ mô hình
model_dir = '/content/drive/MyDrive/Colab_Notebooks/NLP/Week2/model'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Đường dẫn tới dữ liệu huấn luyện
path_data = '/content/drive/MyDrive/Colab_Notebooks/NLP/Week2/datatrain.txt'

def read_data(path):
    train_data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            train_data.append(line.split())
    return train_data

if __name__ == '__main__':
    # Đọc dữ liệu
    train_data = read_data(path_data)

    # Huấn luyện mô hình fastText
    model_fasttext = FastText(vector_size=150, window=10, min_count=2, workers=4, sg=1)
    model_fasttext.build_vocab(corpus_file=path_data)
    model_fasttext.train(corpus_file=path_data, total_examples=model_fasttext.corpus_count, total_words=model_fasttext.corpus_total_words, epochs=10)

    # Lưu mô hình
    model_file = os.path.join(model_dir, "fasttext.model")
    try:
        model_fasttext.save(model_file)
        print(f"Model saved successfully to {model_file}")
    except Exception as e:
        print(f"Error saving model: {e}")


Model saved successfully to /content/drive/MyDrive/Colab_Notebooks/NLP/Week2/model/fasttext.model


In [33]:
from gensim.models import FastText

# Đường dẫn tới mô hình fastText đã lưu
model_path = '/content/drive/MyDrive/Colab_Notebooks/NLP/Week2/model/fasttext.model'

# Load mô hình
model = FastText.load(model_path)

# Ví dụ sử dụng mô hình
word = 'công_nghệ'
similar_words = model.wv.most_similar(word)
print(f"Các từ tương đồng với '{word}':")
for sim_word, sim_score in similar_words:
    print(f"- {sim_word}: {sim_score}")

# Tính độ tương đồng giữa hai từ
word1 = 'máy_tính'
word2 = 'phần_mềm'
similarity = model.wv.similarity(word1, word2)
print(f"Độ tương đồng giữa '{word1}' và '{word2}': {similarity}")

# Nhúng vector của một từ
embedding = model.wv[word]
print(f"Vector biểu diễn của '{word}':")
print(embedding)


Các từ tương đồng với 'công_nghệ':
- công_nghệ_cao: 0.9568533897399902
- công_nghiệp: 0.9311946034431458
- công_tác: 0.8829225897789001
- công_ty_con: 0.8553340435028076
- mỏ: 0.8378849625587463
- công_ty: 0.8351039886474609
- chất_thải: 0.8321520090103149
- xây_dựng: 0.8260173797607422
- xử_lý: 0.8235334157943726
- doanh_nghiệp: 0.8115085959434509
Độ tương đồng giữa 'máy_tính' và 'phần_mềm': 0.7191945910453796
Vector biểu diễn của 'công_nghệ':
[ 0.32453415 -0.407155   -0.11300841  0.0508524  -0.13862889  0.12706842
  0.34817088  0.21037945 -0.04579591 -0.15166341 -0.16493711  0.11099705
 -0.06050544 -0.33607385 -0.12834756 -0.39662936 -0.33456424  0.08054994
 -0.14553684  0.20931049  0.0598273   0.11640912  0.04737476  0.1145734
 -0.13153334 -0.14935179 -0.11864572 -0.3588208   0.18707359  0.02021561
  0.07351134 -0.3637228   0.3099879  -0.06573392 -0.02734224 -0.05424011
 -0.17509072 -0.40064347 -0.01571399 -0.29524663  0.06303214  0.06013732
  0.12795639  0.39757347  0.12265465  0.0